# TELL Runs for IM3's Experiment Group B 

This notebook executes the initial set of runs of the TELL model for IM3's Experiment Group B.

In [1]:
# Start by importing the TELL package and information about your operating system:
import os 
import tell

import pandas as pd
import matplotlib.pyplot as plt

from tell.package_data import get_ba_abbreviations
from tell import state_metadata_from_state_abbreviation
from scipy import interpolate


## Set the Directory Structure

In [2]:
# Identify the top-level directory and the subdirectory where the data will be stored:
current_dir =  '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs_V2'
tell_data_dir = os.path.join(current_dir, r'tell_data')

# If the "tell_data_dir" subdirectory doesn't exist then create it:
if not os.path.exists(tell_data_dir):
   os.makedirs(tell_data_dir)


## Run the MLP Models

In [ ]:
# Generate a list of BA abbreviations to process:
ba_abbrev_list = tell.get_balancing_authority_to_model_dict().keys()

scenario_to_process = 'rcp85hotter_ssp5'

# Run the MLP prediction step for the list of BAs using parallel processing streams:
for year_to_process in range(2049,2050,1):
    pdf = tell.predict_batch(target_region_list = ba_abbrev_list,
                             year = year_to_process,
                             data_dir = os.path.join(tell_data_dir, r'wrf_to_tell_data', scenario_to_process),
                             datetime_field_name = 'Time_UTC',
                             save_prediction = True,
                             prediction_output_directory = os.path.join(tell_data_dir, r'outputs', r'mlp_output', scenario_to_process),
                             n_jobs = -1)


## Test the New Ingest Function for the GCAM-USA Data


In [ ]:
gcam_usa_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs_V2/tell_data/gcamusa_data'


In [ ]:
def extract_gcam_usa_loads(scenario_to_process: str):
    """Extracts the state-level annual loads from a GCAM-USA output file.

    :param scenario_to_process: Scenario to process
    :type scenario_to_process:  str

    :return:                    DataFrame of state-level annual total electricity loads

    """

    # Cluge the scenario for historical runs:
    if scenario_to_process == 'historic':
       scenario_to_process_gcam = 'rcp45cooler_ssp3'
    else:
       scenario_to_process_gcam = scenario_to_process

    # Create the filename for the needed GCAM run:
    filename = (os.path.join(gcam_usa_input_dir, ('electricity_demand_' + scenario_to_process_gcam + '.csv')))
    
    # Load in the raw GCAM-USA output file:
    gcam_usa_df = pd.read_csv(filename, index_col=None, header=0)
    
    # Make a list of all of the states in the "gcam_usa_df":
    states = gcam_usa_df['subRegion'].unique()

    # Loop over the states and interpolate their loads to an annual time step:
    for i in range(len(states)):
    # for i in range(1):

        # Subset to just the data for the state being processed:
        subset_df = gcam_usa_df[gcam_usa_df['subRegion'].isin([states[i]])].copy()

        # Retrieve the state metadata:
        (state_fips, state_name) = state_metadata_from_state_abbreviation(states[i])

        # Linearly interpolate the 5-year loads from GCAM-USA to an annual time step:
        annual_time_vector = pd.Series(range(subset_df['x'].min(), subset_df['x'].max()))
        interpolation_function = interpolate.interp1d(subset_df['x'], subset_df['value'], kind='linear')
        annual_loads = interpolation_function(annual_time_vector)

        # Create an empty dataframe and store the results:
        state_df = pd.DataFrame()
        state_df['Year'] = annual_time_vector.tolist()
        state_df['GCAM_USA_State_Annual_Load_TWh'] = annual_loads
        state_df['State_FIPS'] = state_fips
        state_df['State_Name'] = state_name
        state_df['State_Abbreviation'] = states[i]

        # Aggregate the output into a new dataframe:
        if i == 0:
            gcam_usa_output_df = state_df
        else:
            gcam_usa_output_df = pd.concat([gcam_usa_output_df, state_df])

    return gcam_usa_output_df


In [ ]:
test_df = extract_gcam_usa_loads(scenario_to_process = 'rcp45cooler_ssp3')

test_df


## Run the Forward Execution

In [ ]:
# Run the TELL model forward in time:
for year in range(2007,2020,1):
    summary_df, ba_time_series_df, state_time_series_df = tell.execute_forward(year_to_process = str(year),
                                                                               gcam_target_year = str(year), 
                                                                               scenario_to_process = 'historic',
                                                                               data_output_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs_V2/tell_data/outputs/tell_output',
                                                                               gcam_usa_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs_V2/tell_data/gcamusa_data',
                                                                               map_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs_V2/tell_data/ba_service_territory_data',
                                                                               mlp_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs_V2/tell_data/outputs/mlp_output',
                                                                               pop_input_dir = '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs_V2/tell_data/population_data',
                                                                               save_county_data = False)


Scenario =  historic  , Year =  2007
Elapsed time =  0:01:08.574128
Scenario =  historic  , Year =  2008
Elapsed time =  0:01:08.709681
Scenario =  historic  , Year =  2009
Elapsed time =  0:01:08.322191
Scenario =  historic  , Year =  2010
Elapsed time =  0:01:11.559144
Scenario =  historic  , Year =  2011
Elapsed time =  0:01:11.349524
Scenario =  historic  , Year =  2012
Elapsed time =  0:01:11.145497
Scenario =  historic  , Year =  2013
Elapsed time =  0:01:10.215729
Scenario =  historic  , Year =  2014
Elapsed time =  0:01:08.018387
Scenario =  historic  , Year =  2015
Elapsed time =  0:01:09.152000
Scenario =  historic  , Year =  2016
Elapsed time =  0:01:09.537181
Scenario =  historic  , Year =  2017
Elapsed time =  0:01:08.902471
Scenario =  historic  , Year =  2018
Elapsed time =  0:01:09.155384
Scenario =  historic  , Year =  2019
